# Import

### Librerie

In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.models import Sequential, Model
from keras.layers import Dense, Bidirectional, LSTM, SimpleRNN, Dropout, TimeDistributed, Layer, Input, Conv1D, RepeatVector, MaxPooling1D, Flatten, GlobalMaxPool1D
from sklearn.metrics import *
from keras.optimizers import Adam
from statsmodels.tsa.seasonal import seasonal_decompose

### Dataset

In [2]:
dataset = pd.read_csv('dati_continui_con_id.csv')

In [3]:
# creo una lista di di dataframe dove ogni elemento è l'insieme delle date per utente
utenti = []
# carico elenco utenti per scorrere tutti gli id
elenco_utenti = pd.read_csv('../dataset/userinfo.csv')
elenco_utenti.drop(elenco_utenti.columns[[0, 3]], axis=1, inplace=True)
elenco_utenti.columns = ['user_id', 'timezone', 'sex', 'age', 'height']
elenco_utenti.head()

,user_id,timezone,sex,age,height
0,fe5d7aacd82682e68d94ea1e6aa33a1e858337f7,PDT,Female,39,175
1,88359b8e24f4d6b758fb4566860ba9842fc5b350,UTC,Male,26,179
2,e061ccf5eece921027f618686a5aa2412d753b6b,PDT,Female,27,169
3,2e88aac4e69049d0845a5e2dce661ae8f4f55239,UTC,Male,61,177
4,b4e87340761e4c917db734cd52f56ed9abd0b314,UTC,Male,48,167


In [4]:
dataset.set_index(['user_id','date'], inplace=True)

In [5]:
# inserisco per ogni elemento della lista il dataframe relativo ad un utente presente nel dataset
for i, r in elenco_utenti.iterrows():
    utente = r['user_id']
    if utente in dataset.index:
        utenti.append(dataset.loc[utente])

In [6]:
# elimina stagionalitò
senza_stag = []
stato = 0
for corrente in utenti:
    print(str(stato) + " / 15971")
    stato = stato+1;
    if len(corrente) >=14:
        lista_stag = []
        lista_stag.append(corrente[['steps']].squeeze() - seasonal_decompose(corrente[['steps']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['bedin']].squeeze() - seasonal_decompose(corrente[['bedin']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['bedout']].squeeze() - seasonal_decompose(corrente[['bedout']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['sleep duration']].squeeze() - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['deepduration']].squeeze()  - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        lista_stag.append(corrente[['lightduration']].squeeze() - seasonal_decompose(corrente[['sleep duration']], model='additive', period = 7).seasonal)
        frame = {'steps' : lista_stag[0], 'bedin' : lista_stag[1], 'bedout' : lista_stag[2], 'sleep duration' : lista_stag[3], 'deepduration' : lista_stag[4], 'lightduration' : lista_stag[5]}
        senza_stag.append(pd.DataFrame(frame))

0 / 15971
1 / 15971
2 / 15971
3 / 15971
4 / 15971
5 / 15971
6 / 15971
7 / 15971
8 / 15971
9 / 15971
10 / 15971
11 / 15971
12 / 15971
13 / 15971
14 / 15971
15 / 15971
16 / 15971
17 / 15971
18 / 15971
19 / 15971
20 / 15971
21 / 15971
22 / 15971
23 / 15971
24 / 15971
25 / 15971
26 / 15971
27 / 15971
28 / 15971
29 / 15971
30 / 15971
31 / 15971
32 / 15971
33 / 15971
34 / 15971
35 / 15971
36 / 15971
37 / 15971
38 / 15971
39 / 15971
40 / 15971
41 / 15971
42 / 15971
43 / 15971
44 / 15971
45 / 15971
46 / 15971
47 / 15971
48 / 15971
49 / 15971
50 / 15971
51 / 15971
52 / 15971
53 / 15971
54 / 15971
55 / 15971
56 / 15971
57 / 15971
58 / 15971
59 / 15971
60 / 15971
61 / 15971
62 / 15971
63 / 15971
64 / 15971
65 / 15971
66 / 15971
67 / 15971
68 / 15971
69 / 15971
70 / 15971
71 / 15971
72 / 15971
73 / 15971
74 / 15971
75 / 15971
76 / 15971
77 / 15971
78 / 15971
79 / 15971
80 / 15971
81 / 15971
82 / 15971
83 / 15971
84 / 15971
85 / 15971
86 / 15971
87 / 15971
88 / 15971
89 / 15971
90 / 15971
91 / 1597

696 / 15971
697 / 15971
698 / 15971
699 / 15971
700 / 15971
701 / 15971
702 / 15971
703 / 15971
704 / 15971
705 / 15971
706 / 15971
707 / 15971
708 / 15971
709 / 15971
710 / 15971
711 / 15971
712 / 15971
713 / 15971
714 / 15971
715 / 15971
716 / 15971
717 / 15971
718 / 15971
719 / 15971
720 / 15971
721 / 15971
722 / 15971
723 / 15971
724 / 15971
725 / 15971
726 / 15971
727 / 15971
728 / 15971
729 / 15971
730 / 15971
731 / 15971
732 / 15971
733 / 15971
734 / 15971
735 / 15971
736 / 15971
737 / 15971
738 / 15971
739 / 15971
740 / 15971
741 / 15971
742 / 15971
743 / 15971
744 / 15971
745 / 15971
746 / 15971
747 / 15971
748 / 15971
749 / 15971
750 / 15971
751 / 15971
752 / 15971
753 / 15971
754 / 15971
755 / 15971
756 / 15971
757 / 15971
758 / 15971
759 / 15971
760 / 15971
761 / 15971
762 / 15971
763 / 15971
764 / 15971
765 / 15971
766 / 15971
767 / 15971
768 / 15971
769 / 15971
770 / 15971
771 / 15971
772 / 15971
773 / 15971
774 / 15971
775 / 15971
776 / 15971
777 / 15971
778 / 15971
779 

1365 / 15971
1366 / 15971
1367 / 15971
1368 / 15971
1369 / 15971
1370 / 15971
1371 / 15971
1372 / 15971
1373 / 15971
1374 / 15971
1375 / 15971
1376 / 15971
1377 / 15971
1378 / 15971
1379 / 15971
1380 / 15971
1381 / 15971
1382 / 15971
1383 / 15971
1384 / 15971
1385 / 15971
1386 / 15971
1387 / 15971
1388 / 15971
1389 / 15971
1390 / 15971
1391 / 15971
1392 / 15971
1393 / 15971
1394 / 15971
1395 / 15971
1396 / 15971
1397 / 15971
1398 / 15971
1399 / 15971
1400 / 15971
1401 / 15971
1402 / 15971
1403 / 15971
1404 / 15971
1405 / 15971
1406 / 15971
1407 / 15971
1408 / 15971
1409 / 15971
1410 / 15971
1411 / 15971
1412 / 15971
1413 / 15971
1414 / 15971
1415 / 15971
1416 / 15971
1417 / 15971
1418 / 15971
1419 / 15971
1420 / 15971
1421 / 15971
1422 / 15971
1423 / 15971
1424 / 15971
1425 / 15971
1426 / 15971
1427 / 15971
1428 / 15971
1429 / 15971
1430 / 15971
1431 / 15971
1432 / 15971
1433 / 15971
1434 / 15971
1435 / 15971
1436 / 15971
1437 / 15971
1438 / 15971
1439 / 15971
1440 / 15971
1441 / 15971

2024 / 15971
2025 / 15971
2026 / 15971
2027 / 15971
2028 / 15971
2029 / 15971
2030 / 15971
2031 / 15971
2032 / 15971
2033 / 15971
2034 / 15971
2035 / 15971
2036 / 15971
2037 / 15971
2038 / 15971
2039 / 15971
2040 / 15971
2041 / 15971
2042 / 15971
2043 / 15971
2044 / 15971
2045 / 15971
2046 / 15971
2047 / 15971
2048 / 15971
2049 / 15971
2050 / 15971
2051 / 15971
2052 / 15971
2053 / 15971
2054 / 15971
2055 / 15971
2056 / 15971
2057 / 15971
2058 / 15971
2059 / 15971
2060 / 15971
2061 / 15971
2062 / 15971
2063 / 15971
2064 / 15971
2065 / 15971
2066 / 15971
2067 / 15971
2068 / 15971
2069 / 15971
2070 / 15971
2071 / 15971
2072 / 15971
2073 / 15971
2074 / 15971
2075 / 15971
2076 / 15971
2077 / 15971
2078 / 15971
2079 / 15971
2080 / 15971
2081 / 15971
2082 / 15971
2083 / 15971
2084 / 15971
2085 / 15971
2086 / 15971
2087 / 15971
2088 / 15971
2089 / 15971
2090 / 15971
2091 / 15971
2092 / 15971
2093 / 15971
2094 / 15971
2095 / 15971
2096 / 15971
2097 / 15971
2098 / 15971
2099 / 15971
2100 / 15971

2667 / 15971
2668 / 15971
2669 / 15971
2670 / 15971
2671 / 15971
2672 / 15971
2673 / 15971
2674 / 15971
2675 / 15971
2676 / 15971
2677 / 15971
2678 / 15971
2679 / 15971
2680 / 15971
2681 / 15971
2682 / 15971
2683 / 15971
2684 / 15971
2685 / 15971
2686 / 15971
2687 / 15971
2688 / 15971
2689 / 15971
2690 / 15971
2691 / 15971
2692 / 15971
2693 / 15971
2694 / 15971
2695 / 15971
2696 / 15971
2697 / 15971
2698 / 15971
2699 / 15971
2700 / 15971
2701 / 15971
2702 / 15971
2703 / 15971
2704 / 15971
2705 / 15971
2706 / 15971
2707 / 15971
2708 / 15971
2709 / 15971
2710 / 15971
2711 / 15971
2712 / 15971
2713 / 15971
2714 / 15971
2715 / 15971
2716 / 15971
2717 / 15971
2718 / 15971
2719 / 15971
2720 / 15971
2721 / 15971
2722 / 15971
2723 / 15971
2724 / 15971
2725 / 15971
2726 / 15971
2727 / 15971
2728 / 15971
2729 / 15971
2730 / 15971
2731 / 15971
2732 / 15971
2733 / 15971
2734 / 15971
2735 / 15971
2736 / 15971
2737 / 15971
2738 / 15971
2739 / 15971
2740 / 15971
2741 / 15971
2742 / 15971
2743 / 15971

...

5231 / 15971
5232 / 15971
5233 / 15971
5234 / 15971
5235 / 15971
5236 / 15971
5237 / 15971
5238 / 15971
5239 / 15971
5240 / 15971
5241 / 15971
5242 / 15971
5243 / 15971
5244 / 15971
5245 / 15971
5246 / 15971
5247 / 15971
5248 / 15971
5249 / 15971
5250 / 15971
5251 / 15971
5252 / 15971
5253 / 15971
5254 / 15971
5255 / 15971
5256 / 15971
5257 / 15971
5258 / 15971
5259 / 15971
5260 / 15971
5261 / 15971
5262 / 15971
5263 / 15971
5264 / 15971
5265 / 15971
5266 / 15971
5267 / 15971
5268 / 15971
5269 / 15971
5270 / 15971
5271 / 15971
5272 / 15971
5273 / 15971
5274 / 15971
5275 / 15971
5276 / 15971
5277 / 15971
5278 / 15971
5279 / 15971
5280 / 15971
5281 / 15971
5282 / 15971
5283 / 15971
5284 / 15971
5285 / 15971
5286 / 15971
5287 / 15971
5288 / 15971
5289 / 15971
5290 / 15971
5291 / 15971
5292 / 15971
5293 / 15971
5294 / 15971
5295 / 15971
5296 / 15971
5297 / 15971
5298 / 15971
5299 / 15971
5300 / 15971
5301 / 15971
5302 / 15971
5303 / 15971
5304 / 15971
5305 / 15971
5306 / 15971
5307 / 15971

5878 / 15971
5879 / 15971
5880 / 15971
5881 / 15971
5882 / 15971
5883 / 15971
5884 / 15971
5885 / 15971
5886 / 15971
5887 / 15971
5888 / 15971
5889 / 15971
5890 / 15971
5891 / 15971
5892 / 15971
5893 / 15971
5894 / 15971
5895 / 15971
5896 / 15971
5897 / 15971
5898 / 15971
5899 / 15971
5900 / 15971
5901 / 15971
5902 / 15971
5903 / 15971
5904 / 15971
5905 / 15971
5906 / 15971
5907 / 15971
5908 / 15971
5909 / 15971
5910 / 15971
5911 / 15971
5912 / 15971
5913 / 15971
5914 / 15971
5915 / 15971
5916 / 15971
5917 / 15971
5918 / 15971
5919 / 15971
5920 / 15971
5921 / 15971
5922 / 15971
5923 / 15971
5924 / 15971
5925 / 15971
5926 / 15971
5927 / 15971
5928 / 15971
5929 / 15971
5930 / 15971
5931 / 15971
5932 / 15971
5933 / 15971
5934 / 15971
5935 / 15971
5936 / 15971
5937 / 15971
5938 / 15971
5939 / 15971
5940 / 15971
5941 / 15971
5942 / 15971
5943 / 15971
5944 / 15971
5945 / 15971
5946 / 15971
5947 / 15971
5948 / 15971
5949 / 15971
5950 / 15971
5951 / 15971
5952 / 15971
5953 / 15971
5954 / 15971

6524 / 15971
6525 / 15971
6526 / 15971
6527 / 15971
6528 / 15971
6529 / 15971
6530 / 15971
6531 / 15971
6532 / 15971
6533 / 15971
6534 / 15971
6535 / 15971
6536 / 15971
6537 / 15971
6538 / 15971
6539 / 15971
6540 / 15971
6541 / 15971
6542 / 15971
6543 / 15971
6544 / 15971
6545 / 15971
6546 / 15971
6547 / 15971
6548 / 15971
6549 / 15971
6550 / 15971
6551 / 15971
6552 / 15971
6553 / 15971
6554 / 15971
6555 / 15971
6556 / 15971
6557 / 15971
6558 / 15971
6559 / 15971
6560 / 15971
6561 / 15971
6562 / 15971
6563 / 15971
6564 / 15971
6565 / 15971
6566 / 15971
6567 / 15971
6568 / 15971
6569 / 15971
6570 / 15971
6571 / 15971
6572 / 15971
6573 / 15971
6574 / 15971
6575 / 15971
6576 / 15971
6577 / 15971
6578 / 15971
6579 / 15971
6580 / 15971
6581 / 15971
6582 / 15971
6583 / 15971
6584 / 15971
6585 / 15971
6586 / 15971
6587 / 15971
6588 / 15971
6589 / 15971
6590 / 15971
6591 / 15971
6592 / 15971
6593 / 15971
6594 / 15971
6595 / 15971
6596 / 15971
6597 / 15971
6598 / 15971
6599 / 15971
6600 / 15971

7171 / 15971
7172 / 15971
7173 / 15971
7174 / 15971
7175 / 15971
7176 / 15971
7177 / 15971
7178 / 15971
7179 / 15971
7180 / 15971
7181 / 15971
7182 / 15971
7183 / 15971
7184 / 15971
7185 / 15971
7186 / 15971
7187 / 15971
7188 / 15971
7189 / 15971
7190 / 15971
7191 / 15971
7192 / 15971
7193 / 15971
7194 / 15971
7195 / 15971
7196 / 15971
7197 / 15971
7198 / 15971
7199 / 15971
7200 / 15971
7201 / 15971
7202 / 15971
7203 / 15971
7204 / 15971
7205 / 15971
7206 / 15971
7207 / 15971
7208 / 15971
7209 / 15971
7210 / 15971
7211 / 15971
7212 / 15971
7213 / 15971
7214 / 15971
7215 / 15971
7216 / 15971
7217 / 15971
7218 / 15971
7219 / 15971
7220 / 15971
7221 / 15971
7222 / 15971
7223 / 15971
7224 / 15971
7225 / 15971
7226 / 15971
7227 / 15971
7228 / 15971
7229 / 15971
7230 / 15971
7231 / 15971
7232 / 15971
7233 / 15971
7234 / 15971
7235 / 15971
7236 / 15971
7237 / 15971
7238 / 15971
7239 / 15971
7240 / 15971
7241 / 15971
7242 / 15971
7243 / 15971
7244 / 15971
7245 / 15971
7246 / 15971
7247 / 15971

7818 / 15971
7819 / 15971
7820 / 15971
7821 / 15971
7822 / 15971
7823 / 15971
7824 / 15971
7825 / 15971
7826 / 15971
7827 / 15971
7828 / 15971
7829 / 15971
7830 / 15971
7831 / 15971
7832 / 15971
7833 / 15971
7834 / 15971
7835 / 15971
7836 / 15971
7837 / 15971
7838 / 15971
7839 / 15971
7840 / 15971
7841 / 15971
7842 / 15971
7843 / 15971
7844 / 15971
7845 / 15971
7846 / 15971
7847 / 15971
7848 / 15971
7849 / 15971
7850 / 15971
7851 / 15971
7852 / 15971
7853 / 15971
7854 / 15971
7855 / 15971
7856 / 15971
7857 / 15971
7858 / 15971
7859 / 15971
7860 / 15971
7861 / 15971
7862 / 15971
7863 / 15971
7864 / 15971
7865 / 15971
7866 / 15971
7867 / 15971
7868 / 15971
7869 / 15971
7870 / 15971
7871 / 15971
7872 / 15971
7873 / 15971
7874 / 15971
7875 / 15971
7876 / 15971
7877 / 15971
7878 / 15971
7879 / 15971
7880 / 15971
7881 / 15971
7882 / 15971
7883 / 15971
7884 / 15971
7885 / 15971
7886 / 15971
7887 / 15971
7888 / 15971
7889 / 15971
7890 / 15971
7891 / 15971
7892 / 15971
7893 / 15971
7894 / 15971

In [7]:
utenti = senza_stag

In [8]:
# Funzione normalizzazione
scaler = MinMaxScaler(feature_range=(0,1))
def normalizza(dat):
    df_for_training_scaled = scaler.fit_transform(dat)
    return df_for_training_scaled

In [9]:
# Adesso normalizza ogni dataset
utenti_normalizzati = []
for u in utenti:
    utenti_normalizzati.append(normalizza(u))

In [10]:
len(utenti_normalizzati)

7292

In [11]:
#n_past is the number of step we will look in the past to predict the next target value.
col2pred = 4  # 0:steps, 1:bedin, 2:bedout, 3:sleep duration, 4:deep duration, 5:lightduration
window_size = 7
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            #per ogni feature fa l'append dei precedenti
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])  
            dataY.append(dataset[i,col2pred])
    return np.array(dataX),np.array(dataY)

In [12]:
# crea insieme di train
train_lista = utenti_normalizzati[:7000]

trainX_lista = []
trainY_lista = []

for t in train_lista:
    trainX_temp, trainY_temp = createXY(t,window_size)
    if len(trainX_temp.shape) == 3 and len(trainY_temp.shape) == 1:
        trainX_lista.append(trainX_temp)
        trainY_lista.append(trainY_temp)

trainX = np.concatenate(trainX_lista)
trainY = np.concatenate(trainY_lista)

In [13]:
# crea insieme di test
test_lista = utenti_normalizzati[7000:]

testX_lista = []
testY_lista = []

for t in test_lista:
    testX_temp, testY_temp = createXY(t,window_size)
    if len(testX_temp.shape) == 3 and len(testY_temp.shape) == 1:
        testX_lista.append(testX_temp)
        testY_lista.append(testY_temp)

testX = np.concatenate(testX_lista)
testY = np.concatenate(testY_lista)

In [14]:
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(883858, 7, 6) (38017, 7, 6) (883858,) (38017,)


In [15]:
# features num
features_num = dataset.shape[1]
features_num

6

In [41]:
trainX_cnn = trainX.reshape(trainX.shape[0], trainX.shape[1], trainX.shape[2], 1)


def create_cnn_BiLSTM():
    model = Sequential()
    model.add(Conv1D(filters=256, kernel_size=2, activation='tanh', input_shape=(window_size,features_num)))
    model.add(Conv1D(filters=128, kernel_size=2, activation='tanh'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(RepeatVector(30))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Bidirectional(LSTM(128, activation='tanh')))
    model.add(Dense(100, activation='tanh'))
    model.add(Dense(1))

    opt = Adam(learning_rate=0.0001)
    model.compile(loss='mse', optimizer=opt)
    return model


model = create_cnn_BiLSTM()


In [42]:
history = model.fit(trainX, trainY, epochs = 5, verbose = 1, batch_size = 4096) # 8192

Epoch 1/5
216/216 [==============================] - 33s 114ms/step - loss: 0.0393
Epoch 2/5
216/216 [==============================] - 24s 111ms/step - loss: 0.0333
Epoch 3/5
216/216 [==============================] - 24s 112ms/step - loss: 0.0327
Epoch 4/5
216/216 [==============================] - 24s 113ms/step - loss: 0.0324
Epoch 5/5
216/216 [==============================] - 24s 112ms/step - loss: 0.0323


In [44]:
prediction_train = model.predict(trainX)
#print("prediction\n", prediction_train)
print("\nPrediction Shape-", prediction_train.shape)

27621/27621 [==============================] - 199s 7ms/step

Prediction Shape- (883858, 1)


In [45]:
print(trainY.shape)

(883858,)


In [46]:
prediction_train_copies_array = np.repeat(prediction_train, features_num, axis=-1)
pred_train = scaler.inverse_transform(np.reshape(prediction_train_copies_array, (len(prediction_train), features_num)))[:, col2pred]
true_value_train_copies_array = np.repeat(trainY, features_num, axis=-1)
true_value_train = scaler.inverse_transform(np.reshape(true_value_train_copies_array, (len(trainY), features_num)))[:, col2pred]


In [47]:
def metrics(true_value_train, pred_value_train):
    print("**********TRAIN-SET***********")
    print("MAE: {:.3f}".format(mean_absolute_error(
        true_value_train, pred_value_train)))
    print("MSE: {:.3f}".format(
        mean_squared_error(true_value_train, pred_value_train)))
    print("RMSE: {:.3f}".format(
        mean_squared_error(true_value_train, pred_value_train, squared=True)))

In [48]:
metrics(true_value_train, pred_train)

**********TRAIN-SET***********
MAE: 0.501
MSE: 0.409
RMSE: 0.409
